In [ ]:
# default_exp bert_preprocessing.create_bert_features
import os
%load_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


# Create Bert Features

Module to convert preprocessing function outputs to model input, including

- convert texts to token id using transformers tokenizers
- convert labels to label ids / token ids

## Imports

In [ ]:
# export
import random

import numpy as np
from typing import Callable, Dict, Union, Tuple

from loguru import logger

from m3tl.special_tokens import PREDICT
from m3tl.bert_preprocessing.bert_utils import (create_instances_from_document)
from m3tl.base_params import BaseParams

from transformers import PreTrainedTokenizer



In [ ]:
# export


def convert_labels_to_ids(
        target: dict,
        problem: str,
        problem_type: str,
        label_encoder,
        tokenizer=None,
        decoding_length: int = None,
        custom_label_handling_fn: Callable = None,
        tokenized_inputs: dict = None,
        **kwargs) -> tuple:

    if isinstance(target, dict):
        if len(target) == 1 and 'labels' in target:
            target = target['labels']

    if custom_label_handling_fn is not None:
        res = custom_label_handling_fn(
            target, label_encoder, tokenizer, decoding_length, tokenized_inputs=tokenized_inputs, problem=problem, **kwargs)

    # labelid, label_mask tuple
    if isinstance(res, tuple):
        if res[1] is None:
            return {'{}_label_ids'.format(problem): res[0]}
        return {'{}_mask'.format(problem): res[1], '{}_label_ids'.format(problem): res[0]}
    elif isinstance(res, dict):
        return res

    return {'{}_label_ids'.format(problem): res}


def create_bert_pretraining(problem,
                            inputs_list,
                            label_encoder,
                            params,
                            tokenizer
                            ):
    """Slight modification of original code

    Raises:
        ValueError -- Input format not right
    """
    if not isinstance(inputs_list[0][0], list):
        raise ValueError('inputs is expected to be list of list of list.')

    all_documents = []
    for document in inputs_list:
        all_documents.append([])
        doc, _ = document
        for sentence in doc:
            all_documents[-1].append(tokenizer.tokenize('\t'.join(sentence)))

    all_documents = [d for d in all_documents if d]
    rng = random.Random()
    rng.shuffle(all_documents)

    vocab_words = list(tokenizer.vocab.keys())
    instances = []

    print_count = 0
    return_list = []
    for _ in range(params.dupe_factor):
        for document_index in range(len(all_documents)):
            instances = create_instances_from_document(
                all_documents,
                document_index,
                params.max_seq_len,
                params.short_seq_prob,
                params.masked_lm_prob,
                params.max_predictions_per_seq,
                vocab_words, rng)
            for instance in instances:
                tokens = instance.tokens
                segment_ids = list(instance.segment_ids)

                masked_lm_positions = list(instance.masked_lm_positions)

                next_sentence_label = 1 if instance.is_random_next else 0

                mask_lm_dict = tokenizer(instance.masked_lm_labels,
                                         truncation=False,
                                         is_split_into_words=True,
                                         padding='max_length',
                                         max_length=params.max_predictions_per_seq,
                                         return_special_tokens_mask=False,
                                         add_special_tokens=False)
                input_ids = tokenizer.convert_tokens_to_ids(tokens)
                input_mask = [1 for _ in input_ids]
                masked_lm_ids = mask_lm_dict['input_ids']
                masked_lm_weights = mask_lm_dict['attention_mask']
                masked_lm_positions = masked_lm_positions + \
                    masked_lm_ids[len(masked_lm_positions):]

                assert len(input_ids) == len(
                    segment_ids), (len(input_ids), len(segment_ids))
                assert len(masked_lm_ids) == len(masked_lm_positions), (len(
                    masked_lm_ids), len(masked_lm_positions))

                yield_dict = {
                    "input_ids": input_ids,
                    "input_mask": input_mask,
                    "segment_ids": segment_ids,
                    "masked_lm_positions": masked_lm_positions,
                    "masked_lm_ids": masked_lm_ids,
                    "masked_lm_weights": masked_lm_weights,
                    "next_sentence_label_ids": next_sentence_label
                }

                if print_count < 3:
                    logger.debug('%s : %s' %
                                               ('tokens', ' '.join([str(x) for x in tokens])))
                    for k, v in yield_dict.items():
                        if not isinstance(v, int):
                            logger.debug('%s : %s' %
                                                       (k, ' '.join([str(x) for x in v])))
                    print_count += 1

                return_list.append(yield_dict)
    return return_list


def mask_inputs_for_mask_lm(inp_text: str, tokenizer: PreTrainedTokenizer, mask_prob=0.1, max_length=128, max_predictions_per_seq=20) -> str:
    if not inp_text:
        return None, None
    inp_text = list(inp_text)
    mask_idx = [i for i in range(min(len(inp_text), max_length))
                if random.uniform(0, 1) <= mask_prob]
    if not mask_idx:
        return None, None
    masked_text = [inp_text[i] for i in mask_idx]
    inp_text = [t if i not in mask_idx else '[MASK]' for i,
                t in enumerate(inp_text)]

    tokenized_dict = tokenizer(
        inp_text, None,
        truncation=True,
        max_length=max_length,
        is_split_into_words=True,
        padding=False,
        return_special_tokens_mask=False,
        add_special_tokens=True,
        return_overflowing_tokens=True)

    # create mask lm features
    mask_lm_dict = tokenizer(masked_text,
                             truncation=True,
                             is_split_into_words=True,
                             padding='max_length',
                             max_length=max_predictions_per_seq,
                             return_special_tokens_mask=False,
                             add_special_tokens=False,)

    mask_token_id = tokenizer(
        '[MASK]', add_special_tokens=False, is_split_into_words=False)['input_ids'][0]
    masked_lm_positions = [i for i, input_id in enumerate(
        tokenized_dict['input_ids']) if input_id == mask_token_id]
    # pad masked_lm_positions to max_predictions_per_seq
    if len(masked_lm_positions) < max_predictions_per_seq:
        masked_lm_positions = masked_lm_positions + \
            [0 for _ in range(max_predictions_per_seq -
                              len(masked_lm_positions))]
    masked_lm_positions = masked_lm_positions[:max_predictions_per_seq]
    masked_lm_ids = np.array(mask_lm_dict['input_ids'], dtype='int32')
    masked_lm_weights = np.array(mask_lm_dict['attention_mask'], dtype='int32')
    mask_lm_dict = {'masked_lm_positions': masked_lm_positions,
                    'masked_lm_ids': masked_lm_ids,
                    'masked_lm_weights': masked_lm_weights}

    return tokenized_dict, mask_lm_dict


## Shortcut functions

In [ ]:
# export

def collect_modal_name(inputs: dict) -> list:
    return [k for k in inputs.keys() if 'modal_type' not in k and 'modal_info' not in k]


def infer_modal(inputs: dict) -> dict:
    modal_name_list = collect_modal_name(inputs)
    for modal_name in modal_name_list:
        modal_type_name = '{}_modal_type'.format(modal_name)
        # infer modal type
        if modal_type_name not in inputs:
            inp = inputs[modal_name]

            if isinstance(inp, str):
                inputs[modal_type_name] = 'text'
            elif isinstance(inp, int):
                inputs[modal_type_name] = 'category'
            elif isinstance(inp, list) and len(inp) == 0:
                raise ValueError(
                    'Found empty list in input but modal_type is not provided: {}'.format(inputs))
            # tokenized text
            elif isinstance(inp, list) and isinstance(inp[0], str):
                inputs[modal_type_name] = 'text'
            elif isinstance(inp, list) and isinstance(inp[0], int):
                inputs[modal_type_name] = 'category'
            else:
                inputs[modal_type_name] = 'array'
    return inputs


def unify_inputs(inputs: Union[Dict, str]) -> Dict:
    """
    unify various input format to single one as following:
    {
        'title': 'this is test',
        'image': np.array([1,2,3]),
        'class': 'a',
        'title_modal_type': 'text', 
        'image_modal_type': 'array',
        'class_modal_type': 'category',
        'class_modal_info': 10
    }
    """

    # inputs is single-modal
    if isinstance(inputs, str) or isinstance(inputs, list) or 'a' in inputs:
        return {
            'text': inputs,
            'text_modal_type': 'text'
        }
    inputs = infer_modal(inputs)
    return inputs


In [ ]:
# test pure text input
unify_inputs('this is a test')

# test legacy multimodal input
unify_inputs({'text': 'this is a test',
             'image': np.array([1, 2, 3]), 'class': 0})


{'text': 'this is a test',
 'image': array([1, 2, 3]),
 'class': 0,
 'text_modal_type': 'text',
 'image_modal_type': 'array',
 'class_modal_type': 'category'}

## Modal handling functions

In [ ]:
# export
class NextExampleException(Exception):
    pass


def text_modal_input_handling(
        problem: str,
        modal_name: str,
        modal_inputs: dict,
        target: dict = None,
        params: BaseParams = None,
        tokenizer: PreTrainedTokenizer = None) -> Tuple[dict, dict]:
    is_mask_lm = (
        params.problem_type[problem] == 'masklm'
        if problem in params.problem_type else False
    )
    is_seq = (
        params.problem_type[problem] == 'seq_tag'
        if problem in params.problem_type else False
    )
    # tokenize inputs, now the length is fixed, target == raw_target
    if isinstance(modal_inputs, dict):
        tokens_a = modal_inputs['a']
        tokens_b = modal_inputs['b']
    else:
        tokens_a = modal_inputs
        tokens_b = None

    if is_mask_lm:
        tokenized_dict, mlm_feature_dict = mask_inputs_for_mask_lm(
            tokens_a, tokenizer, mask_prob=params.masked_lm_prob,
            max_length=params.max_seq_len, max_predictions_per_seq=params.max_predictions_per_seq)
        if tokenized_dict is None:
            # hacky approach to continue outer loop
            raise NextExampleException
    else:
        mlm_feature_dict = {}

        tokenized_dict = tokenizer.encode_plus(
            tokens_a, tokens_b,
            truncation=True,
            max_length=params.max_seq_len,
            is_split_into_words=False,
            padding=False,
            return_special_tokens_mask=is_seq,
            add_special_tokens=True,
            return_overflowing_tokens=True,
            return_token_type_ids=True)

    input_ids = tokenized_dict['input_ids']
    segment_ids = tokenized_dict['token_type_ids']
    input_mask = tokenized_dict['attention_mask']

    modal_feature_dict = {
        '{}_input_ids'.format(modal_name): input_ids,
        '{}_mask'.format(modal_name): input_mask,
        '{}_segment_ids'.format(modal_name): segment_ids
    }

    modal_feature_dict.update(mlm_feature_dict)
    return modal_feature_dict, tokenized_dict


def array_modal_input_handling(
        problem: str,
        modal_name: str,
        modal_inputs: dict,
        target: dict = None,
        params: BaseParams = None,
        tokenizer: PreTrainedTokenizer = None) -> dict:
    modal_inputs = np.array(modal_inputs)
    # avoid unbounded var
    tokenized_dict = None

    if len(modal_inputs.shape) == 1:
        modal_inputs = np.expand_dims(modal_inputs, axis=0)
    # maybe we need to add a,b support for multimodal inputs?
    segment_ids = np.zeros(
        modal_inputs.shape[0], dtype=np.int32)
    input_mask = [1]*len(modal_inputs)
    modal_feature_dict = {
        '{}_input_ids'.format(modal_name): modal_inputs,
        '{}_mask'.format(modal_name): input_mask,
        '{}_segment_ids'.format(modal_name): segment_ids}
    return modal_feature_dict


def category_modal_input_handling(
        problem: str,
        modal_name: str,
        modal_inputs: dict,
        target: dict = None,
        params: BaseParams = None,
        tokenizer: PreTrainedTokenizer = None,
        raw_inputs: dict = None) -> dict:

    modal_inputs = np.array(modal_inputs, dtype='int')

    modal_info_name = '{}_modal_info'.format(modal_name)
    if modal_info_name not in raw_inputs:
        raise KeyError(
            "category modal {} modal info not set".format(modal_name))
    params.set_problem_info(
        problem=problem, info_name=modal_info_name, info=raw_inputs[modal_info_name])

    if len(modal_inputs.shape) == 0:
        modal_inputs = np.expand_dims(modal_inputs, axis=0)
    elif len(modal_inputs) > 2:
        raise ValueError(
            'rank of category modal should be less than 2, get:{} of problem {}'.format(
                len(modal_inputs.shape), problem))

    segment_ids = np.zeros(
        modal_inputs.shape[0], dtype=np.int32)
    input_mask = [1]*len(modal_inputs)

    modal_feature_dict = {
        '{}_input_ids'.format(modal_name): modal_inputs,
        '{}_mask'.format(modal_name): input_mask,
        '{}_segment_ids'.format(modal_name): segment_ids}
    return modal_feature_dict


In [ ]:
# test seq_tag
from m3tl.test_base import TestBase
from m3tl.utils import load_transformer_tokenizer
from m3tl.predefined_problems import generate_fake_data
tb = TestBase()
params = tb.params


Adding new problem weibo_fake_ner, problem type: seq_tag
Adding new problem weibo_fake_multi_cls, problem type: multi_cls
Adding new problem weibo_fake_cls, problem type: cls
Adding new problem weibo_masklm, problem type: masklm
Adding new problem weibo_fake_regression, problem type: regression
Adding new problem weibo_fake_vector_fit, problem type: vector_fit
Adding new problem weibo_premask_mlm, problem type: premask_mlm


In [ ]:
problem = 'weibo_fake_ner'
modal_name = 'desc'
modal_inputs = 'this is a test'
target = None
tokenizer = load_transformer_tokenizer(
    tokenizer_name=params.transformer_tokenizer_name, load_module_name=params.transformer_tokenizer_loading)
print(text_modal_input_handling(
    problem=problem, modal_name=modal_name, modal_inputs=modal_inputs, tokenizer=tokenizer, params=params))
fake_data = generate_fake_data('dict')
print(category_modal_input_handling(
    problem=problem, modal_name='inputs_cate', modal_inputs=fake_data[0]['inputs_cate'], tokenizer=tokenizer, params=params, raw_inputs=fake_data[0]))


({'desc_input_ids': [101, 8554, 8310, 143, 10060, 102], 'desc_mask': [1, 1, 1, 1, 1, 1], 'desc_segment_ids': [0, 0, 0, 0, 0, 0]}, {'overflowing_tokens': [], 'num_truncated_tokens': -250, 'input_ids': [101, 8554, 8310, 143, 10060, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'special_tokens_mask': [1, 0, 0, 0, 0, 1], 'attention_mask': [1, 1, 1, 1, 1, 1]})
{'inputs_cate_input_ids': array([0]), 'inputs_cate_mask': [1], 'inputs_cate_segment_ids': array([0], dtype=int32)}


## Create bert multimodal features

In [ ]:

# export

def separate_inputs_labels(inp: dict) -> Tuple[dict, dict]:
    inputs_dict = {k: v for k, v in inp.items() if k.startswith('inputs')}
    labels_dict = {k: v for k, v in inp.items() if k.startswith('labels')}

    inputs_dict = remove_dict_prefix(inputs_dict, key='inputs_')
    labels_dict = remove_dict_prefix(labels_dict, key='labels_')
    # some special handling for labels here
    if not labels_dict:
        labels_dict = None

    elif len(labels_dict) == 1:
        labels_dict = list(labels_dict.values())[0]
    return inputs_dict, labels_dict


def remove_dict_prefix(inp: dict, key='inputs_') -> dict:

    cleaned_cols = [
        c.replace(key, '') for c in inp.keys()]

    return {
        cleaned_column: inp[in_column]
        for cleaned_column, in_column in zip(
            cleaned_cols, inp.keys()
        )
    }


In [ ]:
# export

def _create_multimodal_bert_features(problem,
                                     example_list,
                                     label_encoder,
                                     params,
                                     tokenizer,
                                     mode,
                                     problem_type,
                                     is_seq):
    if problem_type == 'pretrain':
        return create_bert_pretraining(
            problem=problem,
            inputs_list=example_list,
            label_encoder=label_encoder,
            params=params,
            tokenizer=tokenizer
        )

    is_mask_lm = problem_type == 'masklm'

    for example_id, example in enumerate(example_list):
        raw_inputs, raw_target = separate_inputs_labels(example)

        raw_inputs = unify_inputs(raw_inputs)
        modal_name_list = [m for m in collect_modal_name(
            raw_inputs) if m != 'record_id']

        # save problem modal type
        if example_id == 0:
            for modal_name in modal_name_list:
                params.set_problem_info(problem=problem, info_name='{}_modal_type'.format(
                    modal_name), info=raw_inputs['{}_modal_type'.format(modal_name)])
                modal_info_name = '{}_modal_info'.format(modal_name)
                if modal_info_name in raw_inputs:
                    params.set_problem_info(
                        problem=problem, info_name=modal_info_name, info=raw_inputs[modal_info_name])

        return_dict = {}
        if 'record_id' in raw_inputs:
            return_dict['record_id'] = raw_inputs['record_id']
        try:
            for idx, modal_name in enumerate(modal_name_list):

                modal_inputs = raw_inputs[modal_name]
                modal_type = raw_inputs['{}_modal_type'.format(modal_name)]

                target = raw_target
                if modal_type == 'text':
                    modal_feature_dict, tokenized_dict = text_modal_input_handling(
                        problem=problem,
                        modal_name=modal_name,
                        modal_inputs=modal_inputs,
                        target=target,
                        params=params,
                        tokenizer=tokenizer
                    )

                elif modal_type == 'array':
                    modal_feature_dict = array_modal_input_handling(
                        problem=problem,
                        modal_name=modal_name,
                        modal_inputs=modal_inputs,
                        target=target,
                        params=params,
                        tokenizer=tokenizer)
                    tokenized_dict = None
                elif modal_type == 'category':
                    modal_feature_dict = category_modal_input_handling(
                        problem=problem,
                        modal_name=modal_name,
                        modal_inputs=modal_inputs,
                        target=target,
                        params=params,
                        tokenizer=tokenizer,
                        raw_inputs=raw_inputs
                    )
                    tokenized_dict = None
                else:
                    raise ValueError(
                        'Undefined modal type: {}'.format(modal_type))

                # encode labels
                if mode != PREDICT:
                    if not is_mask_lm:
                        custom_label_handling_fn = params.label_handling_fn.get(
                            problem_type, None)
                        label_dict = convert_labels_to_ids(
                            target=target, problem=problem, problem_type=problem_type,
                            label_encoder=label_encoder, tokenizer=tokenizer,
                            decoding_length=params.decode_max_seq_len,
                            custom_label_handling_fn=custom_label_handling_fn,
                            tokenized_inputs=tokenized_dict,
                            modal_name=modal_name,
                            modal_type=modal_type)

                        modal_feature_dict.update(label_dict)
                return_dict.update(modal_feature_dict)

        except NextExampleException:
            continue

        if params.print_data:
            if example_id < 1:
                if isinstance(raw_inputs, dict):
                    for raw_input_name, raw_input in raw_inputs.items():
                        logger.debug('{}: {}'.format(
                            raw_input_name, str(raw_input)[:200]))
                else:
                    logger.debug(str(raw_inputs)[:200])
                for return_key, return_item in return_dict.items():
                    logger.debug('{}: {}'.format(
                        return_key, str(return_item)[:200]))
        yield return_dict


## Wrapper functions

In [ ]:
# export
def create_multimodal_bert_features(problem,
                                    example_list,
                                    label_encoder,
                                    params,
                                    tokenizer,
                                    mode,
                                    problem_type,
                                    is_seq):
    if problem_type == 'pretrain':
        return create_bert_pretraining(
            problem=problem,
            inputs_list=example_list,
            label_encoder=label_encoder,
            params=params,
            tokenizer=tokenizer
        )
    gen = _create_multimodal_bert_features(problem,
                                           example_list,
                                           label_encoder,
                                           params,
                                           tokenizer,
                                           mode,
                                           problem_type,
                                           is_seq)
    return_dict_list = [d for d in gen]
    return return_dict_list


def create_multimodal_bert_features_generator(problem,
                                              example_list,
                                              label_encoder,
                                              params,
                                              tokenizer,
                                              mode,
                                              problem_type,
                                              is_seq):
    if problem_type == 'pretrain':
        return create_bert_pretraining(
            problem=problem,
            inputs_list=example_list,
            label_encoder=label_encoder,
            params=params,
            tokenizer=tokenizer
        )
    gen = _create_multimodal_bert_features(problem,
                                           example_list,
                                           label_encoder,
                                           params,
                                           tokenizer,
                                           mode,
                                           problem_type,
                                           is_seq)
    return gen
